In [1]:
import pandas as pd
import glob, os

In [2]:
directory = "/Users/emiadenuga/Downloads/Portfolio Files/Airline"

csv_files = glob.glob(os.path.join(directory, "*.csv"))

In [3]:
dfs = []

for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)

s = combined_df['FL_DATE'].astype(str).str.strip()

d1 = pd.to_datetime(s, format='%Y-%m-%d', errors='coerce')
d2 = pd.to_datetime(s, format='%m/%d/%Y %I:%M:%S %p', errors='coerce')

df['FL_DATE'] = d1.fillna(d2)

combined_df['FL_DATE'] = d1.fillna(d2)

combined_df['FL_DATE'] = combined_df['FL_DATE'].dt.strftime('%Y-%m-%d')

combined_df.to_csv(os.path.join(directory, "airline_data_2023.csv"), index=False)
print("Saved airline_data_2023.csv")


Saved airline_data_2023.csv


In [42]:
df = pd.read_csv("airline_data_2023.csv")

In [43]:
shape = df.shape
print(shape)

column_names = df.columns
print(column_names)

print(df)

(6847899, 12)
Index(['FL_DATE', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'CRS_DEP_TIME',
       'DEP_TIME', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME',
       'ARR_DELAY_NEW', 'CANCELLED', 'DIVERTED'],
      dtype='object')
            FL_DATE OP_UNIQUE_CARRIER ORIGIN DEST  CRS_DEP_TIME  DEP_TIME  \
0        2023-04-01                9E    ABY  ATL           752     747.0   
1        2023-04-01                9E    ABY  ATL          1251    1247.0   
2        2023-04-01                9E    AEX  ATL          1003    1139.0   
3        2023-04-01                9E    AEX  ATL          1600    1551.0   
4        2023-04-01                9E    ALB  DTW           555     551.0   
...             ...               ...    ...  ...           ...       ...   
6847894  2023-01-31                YX    TUL  LGA           619     613.0   
6847895  2023-01-31                YX    TUL  ORD          1104    2013.0   
6847896  2023-01-31                YX    XNA  LGA           701     704.0   
6847897

In [44]:
#df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], format='%Y-%m-%d', errors='coerce')


columns_dropped = ['CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY_NEW', 'CRS_ARR_TIME', 'ARR_TIME']
df = df.drop(columns=columns_dropped)
df = df.rename(columns={'ARR_DELAY_NEW': 'ARR_DELAY'})


df.to_csv('airline_data_2023_1.csv', index=False)

In [45]:
shape = df.shape
print(shape)

column_names = df.columns
print(column_names)

print(df.head)

(6847899, 7)
Index(['FL_DATE', 'OP_UNIQUE_CARRIER', 'ORIGIN', 'DEST', 'ARR_DELAY',
       'CANCELLED', 'DIVERTED'],
      dtype='object')
<bound method NDFrame.head of            FL_DATE OP_UNIQUE_CARRIER ORIGIN DEST  ARR_DELAY  CANCELLED  \
0       2023-04-01                9E    ABY  ATL       25.0        0.0   
1       2023-04-01                9E    ABY  ATL        0.0        0.0   
2       2023-04-01                9E    AEX  ATL       74.0        0.0   
3       2023-04-01                9E    AEX  ATL        0.0        0.0   
4       2023-04-01                9E    ALB  DTW       10.0        0.0   
...            ...               ...    ...  ...        ...        ...   
6847894 2023-01-31                YX    TUL  LGA        0.0        0.0   
6847895 2023-01-31                YX    TUL  ORD      524.0        0.0   
6847896 2023-01-31                YX    XNA  LGA       27.0        0.0   
6847897 2023-01-31                YX    XNA  LGA      193.0        0.0   
6847898 2023-01-31

In [46]:
print(df.isnull().sum())


FL_DATE                   0
OP_UNIQUE_CARRIER         0
ORIGIN                    0
DEST                      0
ARR_DELAY            104496
CANCELLED                 0
DIVERTED                  0
dtype: int64


In [47]:
df.loc[df['ARR_DELAY'].isna(), ['CANCELLED','DIVERTED']].value_counts()


CANCELLED  DIVERTED
1.0        0.0         87943
0.0        1.0         16552
           0.0             1
Name: count, dtype: int64

In [48]:
df['ARR_DELAY'].describe()



count    6.743403e+06
mean     1.523140e+01
std      5.386250e+01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      9.000000e+00
max      4.405000e+03
Name: ARR_DELAY, dtype: float64

In [49]:
df = df[~(
    df['ARR_DELAY'].isna() &
    (df['CANCELLED'] == 0) &
    (df['DIVERTED'] == 0)
)]

In [50]:
import numpy as np
df['IS_DELAYED'] = np.where(df['ARR_DELAY'] > 15,1,0)
df['IS_SEVERE_DELAYED'] = np.where(df['ARR_DELAY'] > 30,1,0)

mask_non_arrival = (df['CANCELLED'] == 1) | (df['DIVERTED'] == 1)
df.loc[mask_non_arrival, ['IS_DELAYED','IS_SEVERE_DELAYED']] = np.nan


In [51]:
df['MONTH'] = df['FL_DATE'].dt.month


In [52]:
df['ROUTE'] = df['ORIGIN'] + '-' + df['DEST']

In [53]:
print(df.head)

<bound method NDFrame.head of            FL_DATE OP_UNIQUE_CARRIER ORIGIN DEST  ARR_DELAY  CANCELLED  \
0       2023-04-01                9E    ABY  ATL       25.0        0.0   
1       2023-04-01                9E    ABY  ATL        0.0        0.0   
2       2023-04-01                9E    AEX  ATL       74.0        0.0   
3       2023-04-01                9E    AEX  ATL        0.0        0.0   
4       2023-04-01                9E    ALB  DTW       10.0        0.0   
...            ...               ...    ...  ...        ...        ...   
6847894 2023-01-31                YX    TUL  LGA        0.0        0.0   
6847895 2023-01-31                YX    TUL  ORD      524.0        0.0   
6847896 2023-01-31                YX    XNA  LGA       27.0        0.0   
6847897 2023-01-31                YX    XNA  LGA      193.0        0.0   
6847898 2023-01-31                YX    XNA  ORD       41.0        0.0   

         DIVERTED  IS_DELAYED  IS_SEVERE_DELAYED  MONTH    ROUTE  
0             

In [54]:
arrived = df[(df['CANCELLED'] == 0) & (df['DIVERTED'] == 0)].copy()

base = df.groupby(['FL_DATE','OP_UNIQUE_CARRIER','ORIGIN','DEST','ROUTE'], as_index=False).agg(
    FLIGHTS=('ROUTE', 'size'),
    CANCELLED_FLIGHTS=('CANCELLED', 'sum'),
    DIVERTED_FLIGHTS=('DIVERTED', 'sum')
)

arr_perf = arrived.groupby(['FL_DATE','OP_UNIQUE_CARRIER','ORIGIN','DEST','ROUTE'], as_index=False).agg(
    AVG_ARR_DELAY_MIN=('ARR_DELAY', 'mean'),
    DELAY_RATE=('IS_DELAYED', 'mean'),
    SEVERE_DELAY_RATE=('IS_SEVERE_DELAYED', 'mean'),
    ARRIVED_FLIGHTS=('ROUTE', 'size')
)

daily_kpi = base.merge(arr_perf, on=['FL_DATE','OP_UNIQUE_CARRIER','ORIGIN','DEST','ROUTE'], how='left')

daily_kpi['CANCEL_RATE'] = daily_kpi['CANCELLED_FLIGHTS'] / daily_kpi['FLIGHTS']
daily_kpi['DIVERT_RATE'] = daily_kpi['DIVERTED_FLIGHTS'] / daily_kpi['FLIGHTS']


In [55]:
final_columns = [
    'FL_DATE',
    'OP_UNIQUE_CARRIER',
    'ORIGIN',
    'DEST',
    'ROUTE',
    'FLIGHTS',
    'CANCELLED_FLIGHTS',
    'DIVERTED_FLIGHTS',
    'CANCEL_RATE',
    'DIVERT_RATE',
    'AVG_ARR_DELAY_MIN',
    'DELAY_RATE',
    'SEVERE_DELAY_RATE'
]


In [56]:
daily_kpi = daily_kpi[final_columns]
daily_kpi.to_csv("airline_kpi_daily.csv", index=False)


In [57]:
daily_kpi.shape


(2814071, 13)

In [58]:
daily_kpi[['FLIGHTS','CANCEL_RATE','DIVERT_RATE','DELAY_RATE']].describe()
daily_kpi.sort_values('FLIGHTS', ascending=False).head()


,FL_DATE,OP_UNIQUE_CARRIER,ORIGIN,DEST,ROUTE,FLIGHTS,CANCELLED_FLIGHTS,DIVERTED_FLIGHTS,CANCEL_RATE,DIVERT_RATE,AVG_ARR_DELAY_MIN,DELAY_RATE,SEVERE_DELAY_RATE
1704044,2023-08-10,HA,OGG,HNL,OGG-HNL,35,0.0,0.0,0.000000,0.0,15.000000,0.285714,0.200000
1712119,2023-08-11,HA,OGG,HNL,OGG-HNL,31,3.0,0.0,0.096774,0.0,14.035714,0.250000,0.071429
1696022,2023-08-09,HA,OGG,HNL,OGG-HNL,28,0.0,0.0,0.000000,0.0,10.285714,0.214286,0.107143
755198,2023-04-10,HA,OGG,HNL,OGG-HNL,26,0.0,0.0,0.000000,0.0,16.615385,0.423077,0.269231
1313381,2023-06-22,HA,HNL,OGG,HNL-OGG,26,1.0,0.0,0.038462,0.0,6.800000,0.080000,0.080000
